<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Adding-a-field-to-interaction-display-files" data-toc-modified-id="Adding-a-field-to-interaction-display-files-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Adding a field to interaction display files</a></span><ul class="toc-item"><li><span><a href="#Preparation" data-toc-modified-id="Preparation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Preparation</a></span><ul class="toc-item"><li><span><a href="#Simple-sample-generation" data-toc-modified-id="Simple-sample-generation-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Simple sample generation</a></span></li></ul></li><li><span><a href="#Computation" data-toc-modified-id="Computation-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Computation</a></span><ul class="toc-item"><li><span><a href="#Simulation-parameters-definition" data-toc-modified-id="Simulation-parameters-definition-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Simulation parameters definition</a></span></li><li><span><a href="#Computation-with-usual-visualization" data-toc-modified-id="Computation-with-usual-visualization-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Computation with usual visualization</a></span></li><li><span><a href="#Adding-a-field-to-Paraview" data-toc-modified-id="Adding-a-field-to-Paraview-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Adding a field to Paraview</a></span></li></ul></li></ul></li></ul></div>

# Adding a field to interaction display files

It is possible within a __pylmgc90.chipy__ python script, to add new fields to the display files generated by the `chipy.WriteDisplayFiles` function.

But depending on the file to write, the syntax may be a little cumbersome. Here will be presented a
simple case showing how to do this for the `ptc` block describing the interactions.

In the following we prepare a new sample, build a computation script and improve it to add new fields to display files. 


## Preparation

### Simple sample generation

The idea is to simulate a cohesive contact law in traction.
Thus the sample is made only of 3 bricks, the first one fixed in space,
the second one sticked on the first one with a `COUPLED_DOF` contact law
and the last one sticked to the second one with the `IQS_TH_CZM` contact law.
The third one is pulled on with a velocity boundary condition.

<img src="images/mode1.png" alt="Drawing" style="width: 200px;"/>

In [ ]:
from pathlib import Path
import numpy as np

from pylmgc90 import pre

datbox = Path('./DATBOX')
datbox.mkdir(exist_ok=True)

dim = 2

# containers
bodies = pre.avatars()
mats   = pre.materials()
mods   = pre.models()
svs    = pre.see_tables()
tacts  = pre.tact_behavs()

# model
mod = pre.model(name='rigid', physics='MECAx', element='Rxx2D', dimension=dim)
mods.addModel(mod)

# material
tdur = pre.material(name='TDURx',materialType='RIGID',density=1000.)
mats.addMaterial(tdur)

# bricks generation
v = np.array( [ [-0.5,-0.5], [0.5,-0.5], [0.5,0.5], [-0.5,0.5] ] )

# down one
down = pre.rigidPolygon(center=[0.,0.], model=mod, material=tdur, theta=0., color='BLUEx',
                        generation_type='full', vertices=v )
down.imposeDrivenDof(component=[1, 2, 3], dofty='vlocy') # fixed
bodies += down

# center one
center = pre.rigidPolygon(center=[0,1.], model=mod, material=tdur, theta=0., color='BLUEx',
                          generation_type='full', vertices=v )
bodies += center

# upper one
upper = pre.rigidPolygon(center=[0,2.], model=mod, material=tdur, theta=0., color='REDxx',
                         generation_type='full', vertices=v )
# fixed on x-axis and rotation
upper.imposeDrivenDof(component=[1, 3], dofty='vlocy')
# fixed velocity on y-axis
upper.imposeDrivenDof(component=2, dofty='vlocy',description='predefined',ct=0.001,rampi=1.0, ramp=0.0)
bodies += upper

# interaction management
ldc = pre.tact_behav(name='cd___',law='COUPLED_DOF')
tacts += ldc

lcu = pre.tact_behav(name='czm__', law='IQS_TH_CZM', dyfr=0.1, stfr=0.1,
                     cn=1e10, s1=1e5, G1=1., dp1=1.5e-5,
                     ct=1e10, s2=1e5, G2=1., dp2=1.5e-5)
tacts += lcu

svdc = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='POLYG', colorCandidat   ='BLUEx', behav=ldc, alert=0.1,
                     CorpsAntagoniste='RBDY2', antagoniste='POLYG', colorAntagoniste='BLUEx' )
svs += svdc

svcu = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='POLYG', colorCandidat   ='BLUEx', behav=lcu, alert=0.1,
                     CorpsAntagoniste='RBDY2', antagoniste='POLYG', colorAntagoniste='REDxx' )
svs += svcu

# postpro
post = pre.postpro_commands()
my_command=pre.postpro_command(name='SOLVER INFORMATIONS', step=1)
post.addCommand(my_command)

rigid_brick_disp = pre.postpro_command(name='BODY TRACKING', step=1, rigid_set=[down,center,upper])
post.addCommand(rigid_brick_disp)
rigid_brick_torque = pre.postpro_command(name='TORQUE EVOLUTION', step=1, rigid_set=[down,center,upper])
post.addCommand(rigid_brick_torque)

# file writing
pre.writeDatbox(dim, mats, mods, bodies, tacts, svs, post=post, gravy=[0.,0.,0.], datbox_path=datbox)

In [ ]:
pre.visuAvatars(bodies)

## Computation

Compared to previous examples, the example used here needs to:
1. use the `chipy.tact_behav_SetCZMwithInitialFriction` function after the *initiliaze* step,
2. deactivate the default visualization step of the *computation* step.

### Simulation parameters definition

Definition of all the parameters to compute the simulation:

In [ ]:
from pylmgc90 import chipy
from pylmgc90.chipy import computation

import numpy as np

# space dimension
dim = 2

# time evolution parameters
dt = 1e-4
nb_steps = 500

# theta integrator parameter
theta = 0.5

# nlgs parameters
stype = 'Stored_Delassus_Loops         '
norm  = 'Quad '
conv  = 1e-4
relax = 1.0
gsit1 = 500
gsit2 = 10

# write parameter
freq_write   = 5

# display parameters
freq_display = 5

czm_fric = 0.3

### Computation with usual visualization

So, now that the parameters are all defined, an usual computation
is done like this:

In [ ]:
computation.initialize( dim, dt, theta, h5_file='lmgc90.h5' )
chipy.tact_behav_SetCZMwithInitialFriction(czm_fric)

for k in range(1, nb_steps+1):
    if k%30 == 0:
        print( f"computing step {k}" )
    computation.one_step(stype, norm, conv, relax, gsit1, gsit2, freq_write, 0)
    chipy.WriteDisplayFiles(freq_display)

computation.finalize()
print('Computation finished')

You can check the result in paraview (if you run paraview from here,
remember to close it before trying to run new cells).

In [ ]:
#running a specific version of paraview on macos
#!/Applications/ParaView-5.3.0-RC2.app/Contents/MacOS/paraview

#running a specific version of paraview on Linux
#!paraview

### Adding a field to Paraview

In this exercice, it is desired to display the damage field and the vector
of the local velocity in global frame.

For the damage field, and for any internal variable of the interaction, it
is only needed to register the variable and add it to the display field. This
is done with the functions:
 * `chipy.registerInterInternals`
 * `chipy.addRegistersToDisplay`

For the user defined field, the value must be defined for every interactions,
so an array matching the output of `chipy.getInteractions` must be provided.

Now, during the time loop iteration what is needed is:
  * get the current interactions with the `chipy.getInteractions` function.
  * generate the desire *Vl* vector for each interaction from the `uc` (contact
locus frame) and the `vl` field of the interactions array
  * give this *Vl* array to `chipy.WriteDisplayFiles`.

Since each time step may have a different number of interaction, the values array
cannot be created beforehand (unlike when dealing with contactors for example).

In [ ]:
computation.initialize( dim, dt, theta, h5_file='lmgc90.h5' )
chipy.tact_behav_SetCZMwithInitialFriction(czm_fric)
chipy.registerInterInternals('beta')
chipy.addRegistersToDisplay(True)
for k in range(1, nb_steps+1):
    if k%30 == 0:
        print( f"computing step {k}" )
    computation.one_step(stype, norm, conv, relax, gsit1, gsit2, freq_write, 0)
    inters = chipy.getInteractions()
    vl = np.matmul( inters['uc'], inters['vl'][:,:,np.newaxis] )[:,:,0]
    chipy.WriteDisplayFiles(freq_display, Vl=('ptc',vl,))

computation.finalize()
print('Computation finished')

Now check in paraview that the `beta` and the `Vl` (or any other name you choose if you changed it) field
appears in the *ptc* block of the *lmgc90.pvd* display files.

In [ ]:
#running a specific version of paraview on macos
#!/Applications/ParaView-5.3.0-RC2.app/Contents/MacOS/paraview

#running a specific version of paraview on Linux
#!paraview